## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* シグネチャ（LeetCode 想定）:

```python
import pandas as pd

def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    ...
```

* 制約:

  * IO なし
  * `print` / `sort_values` 不使用
* 判定は ID 基準:

  * `product.product_id`（商品マスタ）
  * `sales.product_id`（売上側 FK → マスタ）

---

## 1) 問題

### PROBLEM_STATEMENT

`Sales` と `Product` の 2 つのテーブル（DataFrame）がある。

* **Sales**

  * 各行は、ある年における `product_id` の販売
  * `price` は 1 単位あたりの単価
* **Product**

  * 各 `product_id` ごとの `product_name`

**各 sale_id について**, 対応する `product_name`, `year`, `price` を出力せよ。
結果の行順は任意。

### 入力 DF（INPUT_DATAFRAMES）

* `sales: pd.DataFrame`

  * 列: `['sale_id', 'product_id', 'year', 'quantity', 'price']`
* `product: pd.DataFrame`

  * 列: `['product_id', 'product_name']`

### 出力（OUTPUT_COLUMNS_AND_RULES）

* 戻り値: `pd.DataFrame`
* 列名・順序:

```text
['product_name', 'year', 'price']
```

* 各行は `Sales` の 1 レコードに対応

---

## 2) 実装（指定シグネチャ厳守）

> この問題は **グループ処理不要** なので、
> 「列最小化 → 軽量 `map` で名前付与 → 必要列だけ DataFrame 化」が最短です。

```python
import pandas as pd

def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['product_name', 'year', 'price']
    """
    # 1) Product 側を「product_id → product_name」のマッピングに圧縮
    key_to_name = product.set_index('product_id')['product_name']

    # 2) Sales の product_id をマップして product_name を付与し、
    #    year と price はそのまま流用して新しい DataFrame を構成
    out = pd.DataFrame({
        'product_name': sales['product_id'].map(key_to_name),
        'year': sales['year'],
        'price': sales['price'],
    })

    return out

Analyze Complexity
Runtime 375 ms
Beats 20.66%
Memory 69.10 MB
Beats 99.76%

```

* 列順は dict リテラルの順序どおり → `['product_name', 'year', 'price']`
* `sort_values` / `print` 未使用
* groupby / rank も不要なため使っていない

---

## 3) アルゴリズム説明

使っている主な API は以下の通りです。

1. **`set_index`**

   * `product.set_index('product_id')['product_name']`
   * `product_id` をインデックスにして、`product_name` だけ取り出した `Series` を作成
   * これで「単一キー → 単一値」のマップを構築

2. **`map`**

   * `sales['product_id'].map(key_to_name)`
   * 各 `product_id` に対して、対応する `product_name` を引く
   * PK（`product_id` 一意）前提のため、`merge` より軽量な 1 対 1 置換として最適

3. **DataFrame の再構成**

   * `pd.DataFrame({...})` で必要列のみをまとめる
   * 余計な列（`sale_id`, `quantity` 等）は出力しない

### NULL / 重複 / 型まわり

* **重複**

  * `Product.product_id` は PK なので重複の心配なし
  * よって `map` で「どの値を取るか」問題は発生しない
* **NULL**

  * 正常系では `Sales.product_id` は FK → `Product` 側に必ず存在する前提
  * もし存在しない場合は `product_name` が `NaN` になるが、問題文の前提上考慮不要
* **型**

  * `product_id` は両 DataFrame で `int` 想定
  * 型がズレるとマッピングできないので、実務なら `astype` で揃えるが、LeetCode では前提一致

---

## 4) 計算量（概算）

* `product.set_index('product_id')['product_name']`

  * インデックス構築: **O(M)**
* `sales['product_id'].map(key_to_name)`

  * 各行でハッシュルックアップ: **O(N)** 近似
* DataFrame 再構成: **O(N)**

ここで

* `N = len(sales)`
* `M = len(product)`

よって全体は **O(N + M)**。
`merge` でも実質同程度ですが、「1 対 1 マッピング」前提なら `map` の方がコードもコストも少しだけ軽い実装になります。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 sales DataFrame]
  B[入力 product DataFrame]
  C[product を product_id と product_name に圧縮してマップ作成]
  D[sales の product_id に map で product_name を付与]
  E[year と price をそのまま利用して列を構成]
  F[出力 product_name year price の DataFrame]

  A --> D
  B --> C
  C --> D
  D --> E
  E --> F
```

この問題は、「**Pandas での JOIN 相当処理は `merge` だけでなく `map` も強力**」というパターンを押さえておくと、後続の問題でもかなり使い回せます。

結論から言うと：

> **アルゴリズム的にはほぼ最適**で、375ms / Beats 20% は「悪くないが特別速くもない」くらい。
> 大きく改善する余地は小さいですが、**実装スタイルを少し変えてワンチャン狙う余地**はあります。

---

## 1. 現状の評価

実装（`map` 版）はこんなイメージでした：

```python
key_to_name = product.set_index('product_id')['product_name']

out = pd.DataFrame({
    'product_name': sales['product_id'].map(key_to_name),
    'year': sales['year'],
    'price': sales['price'],
})
```

* 計算量: **O(N + M)**（N = sales 行数, M = product 行数）
* 余計な処理なし（groupby / sort / 重複処理などゼロ）
* メモリ: Beats 99.76% → 実際かなり優秀

Runtime のパーセンタイルが低いのは、

* LeetCode の pandas ランタイムが結構ブレる
* `merge` ベース実装に最適化が効いている可能性

あたりが効いていそうです。

---

## 2. 試す価値がある改善案（`merge` ベース）

pandas の内部実装的には、**`merge`（ハッシュ結合）に最適化が乗りやすい**ことが多いので、
以下のような **列最小化 → merge → 列並べ替え** パターンは試す価値があります。

```python
import pandas as pd

def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['product_name', 'year', 'price']
    """
    # 1) 列最小化（不要な列を抱えない）
    sales_small = sales[['product_id', 'year', 'price']]
    product_small = product[['product_id', 'product_name']]

    # 2) 左結合（Sales ベース）で product_name を付与
    merged = sales_small.merge(
        product_small,
        on='product_id',
        how='left'
    )

    # 3) 仕様どおりの列のみ・順序に整形
    out = merged[['product_name', 'year', 'price']]

    return out

Analyze Complexity
Runtime 341 ms
Beats 62.84%
Memory 70.59 MB
Beats 9.35%

```

### この書き方の狙い

* **列最小化**

  * `sales_small`, `product_small` で「ほんとうに使う列だけ」に絞る
  * 中間 DataFrame の列数が少ないほど、結合やコピーのコストが減る
* **`merge` の内部最適化に乗る**

  * C 実装のハッシュ結合がフル活用されるので、環境によっては `map` より速くなることがある
* **DataFrame の再構成を 1 回にする**

  * `merged[['product_name', 'year', 'price']]` で slice するだけなので、
    dict からの新規 DataFrame 構築よりわずかに有利なパターンもあり得る

---

## 3. もう一つの軽量案（`join` + index）

`set_index` を使うなら、`map` ではなく `join` に寄せるパターンもあります。

```python
import pandas as pd

def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['product_name', 'year', 'price']
    """
    sales_small = sales[['product_id', 'year', 'price']]
    product_idx = product.set_index('product_id')[['product_name']]

    merged = sales_small.join(product_idx, on='product_id')

    out = merged[['product_name', 'year', 'price']]

    return out

Analyze Complexity
Runtime 345 ms
Beats 57.03%
Memory 70.14 MB
Beats 65.28%

```

* `join(..., on='product_id')` は「左側の index を使って右側 index と結合」する仕組み
* これも **O(N + M)** で、`merge` とほぼ同等の計算量

---

## 4. どこまで追うかの目安

* 375ms / Beats 20.66% は、
  pandas 問題としては **「普通〜やや遅めだけど許容範囲」** くらいの位置
* すでに **メモリはほぼ最上位** なので、

  * これ以上メモリを削る意味は薄い
  * もし `merge`/`join` に変えてランタイムが少し良くなればラッキー、程度の世界

正直、**アルゴリズムはもう最適レベル**なので、
ここから先は「スタイル変更で実行エンジンのクセに乗るかどうか」の話になります。

---

### まとめ

* 今の `map` 実装でも十分 OK（アルゴリズム的には改善余地ほぼなし）
* それでも Runtime を追いたいなら：

  * 上記の **`merge` + 列最小化** 版
  * または **`join` + index** 版
    を一度投げてみる価値はあります
* いずれも計算量は同じで、差は「実装スタイル」と「エンジン側の最適化次第」です

「学習として試す」なら `merge` 版を一度投げて、Runtime の変化を見ておくと今後の問題にも効いてきます 💡
